In [1]:
from azureml.core import Datastore, Dataset, Workspace, Experiment, Run

path='C:/Users/satya/Downloads/credit_risk_dataset.csv'

In [3]:
ws=Workspace.create(name='Batchworkspace',location='eastus2',subscription_id='1db33695-8135-4616-9bb4-9574b401d454',
resource_group='batchRG02',create_resource_group=True,exist_ok=True)

In [4]:
key='igt0Fz9vOWNAfROn5BOxOkD8nuwkQzUc1dP7FMyfSXt1qPkxcwlfTP9a6q1opH+b1DnzS+y0zWKy+ASt17OX2g=='
credit_datastore=Datastore.register_azure_blob_container(ws,'creditdatastore',account_key=key, account_name='creditdatastore',container_name='credit')
paths=[(Datastore.get(ws,'creditdatastore'),'credit_risk_dataset.csv')]
credit_dataset=Dataset.Tabular.from_delimited_files(path=paths)
credit_dataset=credit_dataset.register(ws,name='credi_dataset_registered',create_new_version=True)


In [5]:
credit_dataset_df=credit_dataset.to_pandas_dataframe()
credit_dataset_df

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,True,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,False,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,False,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,False,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,True,4
...,...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,0,0.11,False,30
32577,54,120000,MORTGAGE,4.0,PERSONAL,A,17625,7.49,0,0.15,False,19
32578,65,76000,RENT,3.0,HOMEIMPROVEMENT,B,35000,10.99,1,0.46,False,28
32579,56,150000,MORTGAGE,5.0,PERSONAL,B,15000,11.48,0,0.10,False,26


In [78]:
x

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,3,123.0,4,3,35000,16.02,0.59,True,3
1,21,9600,2,5.0,1,1,1000,11.14,0.10,False,2
2,25,9600,0,1.0,3,2,5500,12.87,0.57,False,3
3,23,65500,3,4.0,3,2,35000,15.23,0.53,False,2
4,24,54400,3,8.0,3,2,35000,14.27,0.55,True,4
...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,0,1.0,4,2,5800,13.16,0.11,False,30
32577,54,120000,0,4.0,4,0,17625,7.49,0.15,False,19
32578,65,76000,3,3.0,2,1,35000,10.99,0.46,False,28
32579,56,150000,0,5.0,4,1,15000,11.48,0.10,False,26


In [81]:
x.dtypes

person_age                      int64
person_income                   int64
person_home_ownership           int32
person_emp_length             float64
loan_intent                     int32
loan_grade                      int32
loan_amnt                       int64
loan_int_rate                 float64
loan_percent_income           float64
cb_person_default_on_file        bool
cb_person_cred_hist_length      int64
dtype: object

In [6]:
credit_dataset_df['person_emp_length'].fillna(credit_dataset_df['person_emp_length'].median(),inplace=True)


In [7]:
credit_dataset_df['loan_int_rate'].fillna(credit_dataset_df['loan_int_rate'].median(),inplace=True)

In [8]:
credit_dataset_df.dtypes

person_age                      int64
person_income                   int64
person_home_ownership          object
person_emp_length             float64
loan_intent                    object
loan_grade                     object
loan_amnt                       int64
loan_int_rate                 float64
loan_status                     int64
loan_percent_income           float64
cb_person_default_on_file        bool
cb_person_cred_hist_length      int64
dtype: object

In [9]:
credit_dataset_df['person_home_ownership'].value_counts()

RENT        16446
MORTGAGE    13444
OWN          2584
OTHER         107
Name: person_home_ownership, dtype: int64

In [28]:
%conda install -c conda-forge xgboost

^C

Note: you may need to restart the kernel to use updated packages.


In [67]:
x.head(10)

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,3,123.0,4,3,35000,16.02,0.59,True,3
1,21,9600,2,5.0,1,1,1000,11.14,0.10,False,2
2,25,9600,0,1.0,3,2,5500,12.87,0.57,False,3
3,23,65500,3,4.0,3,2,35000,15.23,0.53,False,2
4,24,54400,3,8.0,3,2,35000,14.27,0.55,True,4
5,21,9900,2,2.0,5,0,2500,7.14,0.25,False,2
6,26,77100,3,8.0,1,1,35000,12.42,0.45,False,3
7,24,78956,3,5.0,3,1,35000,11.11,0.44,False,4
8,24,83000,3,8.0,4,0,35000,8.90,0.42,False,2
9,21,10000,2,6.0,5,3,1600,14.74,0.16,False,3


In [10]:
experiment=Experiment(ws,'batch_inference_credit')
run=experiment.start_logging(snapshot_directory=None)
from sklearn.preprocessing import LabelEncoder
#from xgboost import XGBClassifier
#import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
LE=LabelEncoder()
x=credit_dataset_df.drop('loan_status',axis=1)
y=credit_dataset_df['loan_status']
x['person_home_ownership']=LE.fit_transform(x['person_home_ownership'])
x['loan_intent']=LE.fit_transform(x['loan_intent'])
x['loan_grade']=LE.fit_transform(x['loan_grade'])
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)
#xgb=XGBClassifier()
rfc=RandomForestClassifier()
fitted_model=rfc.fit(x_train,y_train)
y_pred=fitted_model.predict(x_test)
run.log('accuracy',accuracy_score(y_pred,y_test))
confusion_matrix=confusion_matrix(y_test,y_pred)
cmtx=   {
       "schema_type": "confusion_matrix",
       "schema_version": "1.0.0",
       "data": {
           "class_labels": ["0", "1", "2", "3"],
           "matrix": confusion_matrix.tolist()
           
       }
   }
path='./outputs/xgboost_rfc_trained.pkl'
import joblib
joblib.dump(value=fitted_model,filename=path)
run.log_confusion_matrix('confusion_matrix',cmtx)
run.complete()



In [80]:
x['loan_intent']

0        4
1        1
2        3
3        3
4        3
        ..
32576    4
32577    4
32578    2
32579    4
32580    3
Name: loan_intent, Length: 32581, dtype: int32

In [11]:
#register the model to the workspace
run.register_model(model_name='xgboost_trained',model_path='./outputs/xgboost_rfc_trained.pkl',description='xgboost_credit_risk_trained')


Model(workspace=Workspace.create(name='Batchworkspace', subscription_id='1db33695-8135-4616-9bb4-9574b401d454', resource_group='batchRG02'), name=xgboost_trained, id=xgboost_trained:2, version=2, tags={}, properties={})

In [37]:
y_train

23783    0
17672    1
9705     1
24874    1
1623     0
        ..
32511    0
5192     0
12172    1
235      0
29733    1
Name: loan_status, Length: 26064, dtype: int64

In [12]:
ws.get_default_datastore()

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-49bde73d-44bc-4b74-a4fd-85c3b0c77251",
  "account_name": "batchworstorage530b6b230",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [13]:
default_ds=ws.get_default_datastore()
default_ds.upload(src_dir='./batch-data',target_path='batch-data',overwrite=True, show_progress=True)
batch_data=Dataset.File.from_files(path=(default_ds,'batch-data/'))
batch_data=batch_data.register(ws,name='registed_batch_data_credit_test')

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 2 files
Uploading ./batch-data\1.csv
Uploaded ./batch-data\1.csv, 1 files out of an estimated total of 2
Uploading ./batch-data\2.csv
Uploaded ./batch-data\2.csv, 2 files out of an estimated total of 2
Uploaded 2 files


In [14]:
import os
experiment_folder_name='batch_inference_pipeline'
os.makedirs(experiment_folder_name,exist_ok=True)

In [68]:
import pandas as pd
path=['C:/Users/satya/Downloads/Azure/Azure_cp/train_new_new/batch-data/1.csv']
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for i in path:
    t=pd.read_csv(i,header=None)

In [82]:
import numpy as np 
t=np.array(t)
#fitted_model.predict(t)
if 'D' in t:
    t[0][5]=3
elif 'B' in t:
    t[0][5]=1
elif 'C' in t:
    t[[0][5]]=2
elif 'A' in t:
    t[0][5]=0

    
if 'RENT' in t:
    t[0][2]=3
elif 'OWN' in t:
    t[0][2]=2
elif 'MORTGAGE' in t:
    t[0][2]=0
elif 'OTHER' in t:
    t[0][2]=1

if 'PERSONAL' in t:
    t[0][4]=4
elif 'MEDICAL' in t:
    t[0][4]=3
elif 'EDUCATION' in t:
    t[0][4]=1
elif 'VENTURE' in t:
    t[0][4]=5
elif 'DEBTCONSOLIDATION' in t:
    t[0][4]=0
elif 'HOMEIMPROVEMENT' in t:
    t[0][4]=2

if 'Y' in t:
    t[0][9]=True
elif 'N' in t:
    t[0][9]=False
t

array([[22, 59000, 3, 123, 4, 3, 35000, 16.02, 0.59, True, 3]],
      dtype=object)

In [84]:
fitted_model.predict(t.reshape(1,-1))

array([1])

In [ ]:
from azureml.core.model import Model
import numpy as np 
import joblib
import os
def init():
    global model
    model_path=Model.get_model_path('batch_model')
    model=joblib.load(model_path)

def run(mini_batch):
    results=[]
    for f in mini_batch:
        #read comma delimited data into an array
        data=np.genfromtxt(f,delimiter=',')
        prediction=model.predict(data.reshape(1,-1))
        results.append("{}: {}".format(os.path.basename(f),prediction[0]))
    return results

In [15]:
%%writefile $experiment_folder_name/batch_scoring_script.py

from azureml.core.model import Model
import joblib
import pandas as pd
import numpy as np 
def init():
    global model
    model_path=Model.get_model_path('xgboost_trained')
    model=joblib.load(model_path)

def run(mini_batch):
   preds=[]
   for batch in mini_batch:
        batch=pd.read_csv(batch,header=None)
        t=np.array(batch)
        #fitted_model.predict(t)
        if 'D' in t:
           t[0][5]=3
        elif 'B' in t:
           t[0][5]=1
        elif 'C' in t:
           t[[0][5]]=2
        elif 'A' in t:
           t[0][5]=0
        if 'RENT' in t:
           t[0][2]=3
        elif 'OWN' in t:
           t[0][2]=2
        elif 'MORTGAGE' in t:
           t[0][2]=0
        elif 'OTHER' in t:
           t[0][2]=1
        if 'PERSONAL' in t:
           t[0][4]=4
        elif 'MEDICAL' in t:
           t[0][4]=3
        elif 'EDUCATION' in t:
           t[0][4]=1
        elif 'VENTURE' in t:
           t[0][4]=5
        elif 'DEBTCONSOLIDATION' in t:
           t[0][4]=0
        elif 'HOMEIMPROVEMENT' in t:
           t[0][4]=2
        if 'Y' in t:
           t[0][9]=True
        elif 'N' in t:
           t[0][9]=False
        preds.append(model.predict(t.reshape(1,-1)))
   return preds




Overwriting batch_inference_pipeline/batch_scoring_script.py


In [144]:
mini_batch=['C:/Users/satya/Downloads/Azure/Azure_cp/train_new_new/batch-data/1.csv','C:/Users/satya/Downloads/Azure/Azure_cp/train_new_new/batch-data/2.csv']
preds=[]
for batch in mini_batch:
        batch=pd.read_csv(batch,header=None)
        t=np.array(batch)
        #fitted_model.predict(t)
        if 'D' in t:
           t[0][5]=3
        elif 'B' in t:
           t[0][5]=1
        elif 'C' in t:
           t[[0][5]]=2
        elif 'A' in t:
           t[0][5]=0
        if 'RENT' in t:
           t[0][2]=3
        elif 'OWN' in t:
           t[0][2]=2
        elif 'MORTGAGE' in t:
           t[0][2]=0
        elif 'OTHER' in t:
           t[0][2]=1
        if 'PERSONAL' in t:
           t[0][4]=4
        elif 'MEDICAL' in t:
           t[0][4]=3
        elif 'EDUCATION' in t:
           t[0][4]=1
        elif 'VENTURE' in t:
           t[0][4]=5
        elif 'DEBTCONSOLIDATION' in t:
           t[0][4]=0
        elif 'HOMEIMPROVEMENT' in t:
           t[0][4]=2
        if 'Y' in t:
           t[0][9]=True
        elif 'N' in t:
           t[0][9]=False
fitted_model.predict(t.reshape(1,-1))



array([0])

In [31]:
%%writefile $experiment_folder_name/environment.yml
name: batch_environment
dependencies:
- python=3.6.2
- scikit-learn
- xgboost=1.7.1
- joblib
- pip: 
    - azureml-defaults
    - pandas
    - numpy 

Overwriting batch_inference_pipeline/environment.yml


In [16]:
from azureml.core import conda_dependencies, Environment
env=Environment('batch_environment')
dep=conda_dependencies.CondaDependencies.create(pip_packages=['azureml-defaults','pandas','numpy'],conda_packages=['scikit-learn','joblib'])
env.python.conda_dependencies=dep
env.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220616.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "batch_environment",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-for

In [107]:
experiment_folder_name+'/environment.yml'

'batch_inference_pipeline/environment.yml'

In [32]:
from azureml.core import Environment
environment=Environment.from_conda_specification(name='batch_environment',file_path=experiment_folder_name+'/environment.yml')
environment.register(ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220616.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "batch_environment",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "dependencies": [
                "python=3.6.2",
                "s

In [17]:
from azureml.core.compute import AmlCompute
name='MLcluster'
config=AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
cluster=AmlCompute.create(ws,name,config)
cluster.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [122]:
experiment_folder_name

'batch_inference_pipeline'

In [55]:
batch_data

{
  "source": [
    "('workspaceblobstore', 'batch-data/')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "f7ef023d-9b0a-4410-a06f-7bb1f27ab901",
    "name": "registed_batch_data_credit_test",
    "version": 1,
    "workspace": "Workspace.create(name='Batchworkspace', subscription_id='1db33695-8135-4616-9bb4-9574b401d454', resource_group='batchRG01')"
  }
}

In [18]:
#import outputfileconfig 
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
output=PipelineData(name='inference')

parallel_config=ParallelRunConfig(
    source_directory=experiment_folder_name,
    entry_script='batch_scoring_script.py',
    mini_batch_size='1',
    error_threshold=10,
    output_action='append_row',
    environment=env,
    compute_target=cluster,
    node_count=1
)

parrallel_step=ParallelRunStep(
    name='batch_inference',
    parallel_run_config=parallel_config,
    inputs=[batch_data.as_named_input('raw_data')],
    arguments=[],
    output=output,
    allow_reuse=True
)

In [19]:
from azureml.pipeline.core import Pipeline
batch_pipeline_credit=Pipeline(ws,steps=[parrallel_step])
#batch_pipeline_credit.validate()

In [20]:
pipeline_run=batch_pipeline_credit.submit('batch_inference_pipeline')
pipeline_run.wait_for_completion(show_output=True)

Created step batch_inference [b18bd9b0][692a9c68-0234-45ff-a4d6-f3fa6c50501b], (This step will run and generate new outputs)
Submitted PipelineRun ce28d378-3a36-4ea0-9a00-836a05551419
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/ce28d378-3a36-4ea0-9a00-836a05551419?wsid=/subscriptions/1db33695-8135-4616-9bb4-9574b401d454/resourcegroups/batchRG02/workspaces/Batchworkspace&tid=6ad91895-de06-485e-bc51-fce126cc8530
PipelineRunId: ce28d378-3a36-4ea0-9a00-836a05551419
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/ce28d378-3a36-4ea0-9a00-836a05551419?wsid=/subscriptions/1db33695-8135-4616-9bb4-9574b401d454/resourcegroups/batchRG02/workspaces/Batchworkspace&tid=6ad91895-de06-485e-bc51-fce126cc8530
PipelineRun Status: Running


StepRunId: bf9030a3-f370-4134-8575-36a35a9c3a7c
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/bf9030a3-f370-4134-8575-36a35a9c3a7c?wsid=/subscriptions/1db33695-8135-4616-9bb4-9574b401d454/resourcegroups/batchRG02/

'Finished'

In [26]:
prediction_run=next(pipeline_run.get_children())
prediction_output=prediction_run.get_output_data('inference')
prediction_output.download('outputs')

1